In [10]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import sys
import importlib
import numpy as np
import pandas as pd
from pathlib import Path
import h5py
import hdf5storage # conda install hdf5storage

from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.core import DataWriter, NeuronType, Neurons, BinnedSpiketrain, Mua, ProbeGroup, Position, Epoch, Signal, Laps, FlattenedSpiketrains


from pyphocorehelpers.print_helpers import print_keys_if_possible, debug_dump_object_member_shapes
# # import PhoPositionalData as pdp
# # from pyphoplacecellanalysis.PhoPositionalData. import load_exported, process_data
# from pyphoplacecellanalysis.PhoPositionalData.load_exported import *
# # from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import *
# from pyphoplacecellanalysis.PhoPositionalData.plot_data import *
# from pyphoplacecellanalysis.PhoPositionalData.import_data import *
# from pyphoplacecellanalysis.PhoPositionalData.load_exported import import_mat_file
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, extract_spike_timeseries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## Drastically improve Python Tracebacks on exceptions:
# from rich.traceback import install
# install()

## Automatic debugger on exception raised
%pdb on

Automatic pdb calling has been turned ON


In [3]:
import pandas as pd
from pandas.core.resample import TimedeltaIndexResampler
# pd.core.resample.TimedeltaIndexResampler

pandas.core.resample.TimedeltaIndexResampler

In [5]:
# RoyMaze1:
# mat_import_parent_path = Path(r'R:\data\RoyMaze1')
mat_import_parent_path = Path(r'R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1')
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1') # Old one
# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'
# all_vars = load_position_spikes_extras_mats(parent_path=mat_import_parent_path)
all_vars = HiroDataSessionFormatRegisteredClass._load_all_mats(parent_path=mat_import_parent_path)
# all_vars

In [11]:
session = HiroDataSessionFormatRegisteredClass.build_session(basedir=mat_import_parent_path)
session

DataSession(config: <SessionConfig: {'basepath': WindowsPath('R:/rMBP Python Repos 2022-07-07/PhoNeuronGillespie2021CodeRepo/PhoMatlabDataScripting/ExportedData/RoyMaze1'), 'session_name': 'RoyMaze1', 'session_spec': <SessionFolderSpec: {'required_files': [], 'optional_files': [], 'additional_validation_requirements': []};>, 'is_resolved': True, 'resolved_required_filespecs_dict': {}, 'resolved_optional_filespecs_dict': {}};>): Not yet configured.

In [7]:
# all_vars.extras
# print_keys_if_possible('pos', all_vars.pos)
print_keys_if_possible('extras', all_vars.extras)
# print_keys_if_possible('all_vars', all_vars)
# debug_dump_object_member_shapes(all_vars)

- extras: <class 'neuropy.utils.dynamic_container.DynamicContainer'>
	- behavioral_periods: <class 'pandas.core.frame.DataFrame'> - (668, 6)
	- behavioral_epochs: <class 'pandas.core.frame.DataFrame'> - (3, 6)
	- num_periods: <class 'int'>


In [9]:
type(all_vars.extras.behavioral_epochs) # pandas.core.frame.DataFrame
print(list(all_vars.extras.behavioral_epochs.columns)) # ['epoch_index', 'start_seconds_absolute', 'end_seconds_absolute', 'start_seconds', 'end_seconds', 'duration']
all_vars.extras.behavioral_epochs

,epoch_index,start_seconds_absolute,end_seconds_absolute,start_seconds,end_seconds,duration
0,0.0,68368.714228,79168.671561,0.000000,10799.957333,10799.957333
1,1.0,79368.012194,90198.911394,10999.297966,21830.197166,10830.899200
2,2.0,92919.765028,103719.773561,24551.050800,35351.059333,10800.008533


In [12]:
## Adds Session.paradigm (Epochs)
session_absolute_start_timestamp = all_vars.extras.behavioral_epochs.loc[0, 'start_seconds_absolute'] # 68368.714228
session.config.absolute_start_timestamp = session_absolute_start_timestamp
# 'start_seconds' and 'end_seconds' are relative to start
if 'label' in all_vars.extras.behavioral_epochs.columns:
    epoch_labels = all_vars.extras.behavioral_epochs['label']
else:
    num_rows = all_vars.extras.behavioral_epochs.shape[0]
    epoch_labels = [f'epoch{i}' for i in np.arange(num_rows)]
epochs_df = pd.DataFrame({'start':all_vars.extras.behavioral_epochs['start_seconds'].to_numpy(),'stop':all_vars.extras.behavioral_epochs['end_seconds'].to_numpy(), 'label':epoch_labels})
session.paradigm = Epoch(epochs=epochs_df)
session.paradigm

In [18]:
## Adds Positions:
position_sampling_rate_Hz = 1.0 / np.nanmean(np.diff(all_vars.pos.t)) # 1.0/0.03336651239320582 = 29.97016853950917 Hz
session.config.position_sampling_rate_Hz = position_sampling_rate_Hz
# session.position = Position(traces=np.vstack((x, y)), computed_traces=np.full([1, num_samples], np.nan), t_start=active_t_start, sampling_rate=position_sampling_rate_Hz)
t_rel = all_vars.pos.t
x = all_vars.pos.x
y = all_vars.pos.y
session.position = Position.from_separate_arrays(t_rel, x, y)
session.position

In [19]:
## Adds Spikes:
all_vars.spikes.keys


all_vars.spikes
'spike_cells'


c:\users\pho\repos\neuropy\neuropy\core\position.py:315: UserWarning: registration of accessor <class 'neuropy.core.position.PositionAccessor'> under name 'position' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PositionAccessor(PositionDimDataMixin, PositionComputedDataMixin, TimeSlicedMixin):


DynamicContainer({'spike_matrix': array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 2., 2., ..., 0., 0., 3.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 3., 3., 3.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'spike_cells': array([array([[1.00000000e+00, 3.12156667e+00],
              [1.00000000e+00, 3.23186667e+00],
              [1.00000000e+00, 7.65390000e+00],
              ...,
              [1.00000000e+00, 3.53453575e+04],
              [1.00000000e+00, 3.53498584e+04],
              [1.00000000e+00, 3.53500207e+04]]),
       array([[2.00000000e+00, 3.19666667e-02],
              [2.00000000e+00, 5.85333333e-02],
              [2.00000000e+00, 8.36333333e-02],
              ...,
              [2.00000000e+00, 3.53507447e+04],
              [2.00000000e+00, 3.53507492e+04],
              [2.00000000e+00, 3.53508930e+04]]),
       array([[3.00000000e+00, 2.87366667e-01],
              [3.00000000e+00, 8.15900000e-

In [ ]:
'num_cells': 86, 'spike_list', 'reverse_cellID_idx_lookup_map'


In [ ]:
## IDEA: Define a series of fallback options for each variable using the command chain pattern.